In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
!pip install numpy==1.23.5 xgboost pandas==1.5.3 joblib==1.3.2 scikit-learn==1.3.2 tsfresh==0.20.1 category_encoders==2.6.2 scikit-optimize==0.9.0 statsmodels==0.13.5 scipy==1.10.1 tqdm==4.66.1 distributed==2023.3.1 dask==2023.3.1 shap

# Import all required libraries

In [ ]:
import math
import pandas as pd
import numpy as np
import re
import os
import joblib
import sys
import sklearn
import skopt
import shap
import category_encoders
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump, load
from datetime import datetime
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import xgboost as xgb

import tsfresh
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import StratifiedShuffleSplit

from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.utilities.dataframe_functions import roll_time_series

import warnings
warnings.filterwarnings("ignore")

In [ ]:
print("Python version",sys.version)
print("Pandas version", pd.__version__)
print("Numpy version", np.__version__)
print("Scikit-Learn version", sklearn.__version__)
print("Scikit-Optimize version", skopt.__version__)
print("XGBoost version", xgb.__version__)
print("Category Encoders version", category_encoders.__version__)
print("Joblib version", joblib.__version__)
print("Ts fresh version", tsfresh.__version__)

# UTILITY Function 1

In [ ]:
def convert_to_float(df):
    return df.apply(pd.to_numeric, errors='coerce').astype(np.float32)

# UTILITY Function 2

In [ ]:
def convert_yyyywk_to_date(yyyywk):
    number_of_days_in_month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
    time = str(yyyywk)
    year = int(time[:4])
    week_no = int(time[4:6])
   
    total_days = (week_no*7) - 3
    days_count = 0
    cummulative_sum = []
    cummulative_sum.append(0)
 
    for key,val in number_of_days_in_month.items():
        days_count += val
        if year%4 == 0 and key==2:
            days_count+=1
        cummulative_sum.append(days_count)
 
    first = 0
    second = 1
    month = 12
 
    while second < len(cummulative_sum):
        if (cummulative_sum[first] <= total_days) and (total_days <= cummulative_sum[second]):
            month = first+1
            day = total_days-cummulative_sum[first]
            break
        else:
            first+=1
            second+=1
 
    if month<10:
        month_date = str(year)+"-0"+str(month)+"-01"
    else:
        month_date = str(year)+"-"+str(month)+"-01"
    return month_date

In [ ]:
def convert_yyyywk_to_actual_mid_date(yyyywk):
    number_of_days_in_month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
    time = str(yyyywk)
    year = int(time[:4])
    week_no = int(time[4:6])
   
    total_days = (week_no*7) - 3
    days_count = 0
    cummulative_sum = []
    cummulative_sum.append(0)
 
    for key,val in number_of_days_in_month.items():
        days_count += val
        if year%4 == 0 and key==2:
            days_count+=1
        cummulative_sum.append(days_count)
 
    first = 0
    second = 1
    month = 12
 
    while second < len(cummulative_sum):
        if (cummulative_sum[first] <= total_days) and (total_days <= cummulative_sum[second]):
            month = first+1
            day = total_days-cummulative_sum[first]
            break
        else:
            first+=1
            second+=1
    
    day_no="00"
    
    if day <10:
        day_no = "0"+str(day)
    else:
        day_no = str(day)
        
    if month<10:
        month_date = str(year)+"-0"+str(month)+"-"+day_no
    else:
        month_date = str(year)+"-"+str(month)+"-"+day_no
    return month_date

In [ ]:
def get_days_diff(yyyywk, last_yyyywk):
    curr = pd.to_datetime(convert_yyyywk_to_actual_mid_date(yyyywk))
    last = pd.to_datetime(convert_yyyywk_to_actual_mid_date(last_yyyywk))
    #print(curr, last)
    days_diff = (last- curr) / np.timedelta64(1, 'D')
    print(yyyywk, last_yyyywk, days_diff)
    return days_diff

In [ ]:
get_days_diff(202318, 202401)

# UTILITY Function 3

In [ ]:
def convert_date_to_yyyywk(d):
        
    number_of_days_in_month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
    date = pd.to_datetime(d)
    month = date.month
    year = date.year
    day = date.day
    week_number = 1
    days_count = 0
    cummulative_sum = []
    cummulative_sum.append(0)
 
    for key,val in number_of_days_in_month.items():
        days_count += val
        if year%4 == 0 and key==2:
            days_count+=1
        cummulative_sum.append(days_count)
    if month<13 and month>0:
        week_number = math.ceil((cummulative_sum[month-1]+day)/7)
        
    yyyywk="000000"
    if week_number>9:
        yyyywk = str(year)+str(week_number)
    else:
        yyyywk = str(year)+"0"+str(week_number)
    ans = int(yyyywk)    
    return ans

# UTILITY Function 4

In [ ]:
def engineer_timeseries_cols_using_tsfresh_for_live_customers(only_features):
    
    final_features = pd.DataFrame()
    
    # Store Dataframe into CSV files for each customer id and dates

    only_features['YYYYWK'] = only_features['YYYYWK'].astype(int)
    only_features['CUST_ACCOUNT_NUMBER'] = only_features['CUST_ACCOUNT_NUMBER'].astype(int)
    only_features = only_features.loc[:,~only_features.columns.str.contains('^Unnamed', case=False)]
    only_features = only_features.fillna(0.0)

    common_cust_id = only_features['CUST_ACCOUNT_NUMBER'].unique()
    start_time_begin = time.time()
    
    count=1
    for id_ in common_cust_id: 
        print(count)
        start_time = time.time()
        flag=False
        
        print(id_)    

        only_ts_df = only_features[only_features['CUST_ACCOUNT_NUMBER'] == id_]
        
        average_df = only_ts_df.groupby('YYYYWK')[['INVOICE_REVLINE_TOTAL', 'ORIGINAL_AMOUNT_DUE', 'FUNCTIONAL_AMOUNT', 'USED_STORAGE_MB', 'DOCUMENTS_OPENED']].sum()
        
        average_df.reset_index(inplace=True)
        average_df['CUST_ACCOUNT_NUMBER'] = id_

        average_df['YYYYWK'] = average_df['YYYYWK'].astype(int)
        average_df['CUST_ACCOUNT_NUMBER'] = average_df['CUST_ACCOUNT_NUMBER'].astype(int)
    
        #df_test = session.create_dataframe(average_df)
        #df_test.write.copy_into_location("@PS_DOCUWARE_CHURN/only_ts_df"+str(id_)+".csv", file_format_type="csv", format_type_options={"COMPRESSION": "NONE"}, header=True, overwrite=True )
        
        only_ts_df = average_df
        
        if only_ts_df.shape[0] > 1:
            df_rolled = roll_time_series(only_ts_df, column_id="CUST_ACCOUNT_NUMBER", column_sort="YYYYWK")          # roll_time_series
        elif only_ts_df.shape[0] == 1:
            flag=True
            df_rolled = only_ts_df
            df_rolled['id'] = "("+str(only_ts_df.loc[only_ts_df.index[0],'CUST_ACCOUNT_NUMBER'])+", "+str(only_ts_df.loc[only_ts_df.index[0], 'YYYYWK'])+")"
        else:
            continue
        
        columns_to_dropped = set()

        df_rolled_ = df_rolled.loc[:,~df_rolled.columns.str.contains('^(Unnamed|CUST_|YYYYWK|id)', case=False)]
    
        df_rolled_imputed = df_rolled_
        
        df_rolled_imputed['YYYYWK'] = df_rolled['YYYYWK'].to_list()
        df_rolled_imputed['ID'] = df_rolled['id'].to_list()
        df_rolled_imputed['CUST_ACCOUNT_NUMBER'] = df_rolled['CUST_ACCOUNT_NUMBER'].to_list()
        #df_rolled_imputed.rename(columns={'USED_STORAGE__MB':'USED_STORAGE_MB'}, errors="raise",inplace=True)
        
        columns_to_dropped = set()

        df_rolled_imputed = df_rolled_imputed.loc[:,~df_rolled_imputed.columns.str.contains('^Unnamed', case=False)]
    
        extraction_settings = ComprehensiveFCParameters()

        if df_rolled_imputed.shape[0] > 0:
            features = extract_features(df_rolled_imputed, column_id='ID', column_sort='YYYYWK', default_fc_parameters=extraction_settings) #extract_features
            
            df_temp = (features.sort_index().iloc[features.shape[0]-1, :]).to_frame().transpose()
            df_temp['CUST_ACCOUNT_NUMBER'] = id_
        
            final_features = pd.concat([final_features, df_temp], axis=0, ignore_index=True)
            
        del only_ts_df
        del features
        del df_rolled
        del df_rolled_
        del df_rolled_imputed
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        #print(f"for Cust ID = {id_} Elapsed time: {elapsed_time} seconds")
        count+=1
        #if count == 11:
        #    break
    end_time_begin = time.time()
    elapsed_time_begin = end_time_begin - start_time_begin
    #print(f"Total Elapsed time: {elapsed_time_begin} seconds")

    #Remove unwanted columns
    final_features_columns = final_features.columns
    columns_to_be_dropped_from_ts_df = []

    for col in final_features_columns:
        if re.search(r"^(Unnamed:|obr_|prtar|CONTRACT_LINE_ITEMS|PROBABILITY_OF_DELINQUENCY|RICOH_CUSTOM_RISK_MODEL|CUST_ACCOUNT_NUMBER_)", col):
            columns_to_be_dropped_from_ts_df.append(col)

    final_features_filtered = final_features.drop(columns_to_be_dropped_from_ts_df, axis=1)
    
    return final_features_filtered

# UTILITY Function 5

In [ ]:
def engineer_timeseries_cols_using_tsfresh(only_features):
    
    final_features = pd.DataFrame()
    final_calculated = pd.DataFrame()
    # Store Dataframe into CSV files for each customer id and dates

    only_features['YYYYWK'] = only_features['YYYYWK'].astype(int)
    only_features['CUST_ACCOUNT_NUMBER'] = only_features['CUST_ACCOUNT_NUMBER'].astype(int)
    only_features = only_features.loc[:,~only_features.columns.str.contains('^Unnamed', case=False)]
    only_features = only_features.fillna(0.0)

    common_cust_id = only_features['CUST_ACCOUNT_NUMBER'].unique()
    start_time_begin = time.time()
    
    count=1
    for id_ in common_cust_id: 
        print(count)
        start_time = time.time()
        flag=False
        
        print(id_)    
        calculated = pd.DataFrame(columns=['DAYS_REMAINING'])
        only_ts_df = only_features[only_features['CUST_ACCOUNT_NUMBER'] == id_]
        filename = "@PS_DOCUWARE_CHURN/TSFRESH/all_feature_TSFRESH/only_ts_df_"+str(id_)+".csv"
        curr_df = session.create_dataframe(only_ts_df)
        #curr_df.write.copy_into_location("@PS_DOCUWARE_CHURN/TSFRESH/all_feature_TSFRESH/only_ts_df"+str(id_)+".csv", file_format_type="csv", format_type_options={"COMPRESSION": "NONE"}, header=True, overwrite=True )
        #curr_df.write.csv(filename, overwrite=True, single=True)
        #session.file.put(only_ts_df, ,overwrite=True)
        
       
        #print( calculated.shape )
        sum_df = only_ts_df.groupby('YYYYWK')[['INVOICE_REVLINE_TOTAL', 'ORIGINAL_AMOUNT_DUE', 'FUNCTIONAL_AMOUNT', 'USED_STORAGE_MB', 'DOCUMENTS_OPENED']].sum()
        
        sum_df.reset_index(inplace=True)
        sum_df['CUST_ACCOUNT_NUMBER'] = id_

        sum_df['YYYYWK'] = sum_df['YYYYWK'].astype(int)
        sum_df['CUST_ACCOUNT_NUMBER'] = sum_df['CUST_ACCOUNT_NUMBER'].astype(int)
    
        #df_test = session.create_dataframe(sum_df)
        #df_test.write.copy_into_location("@PS_DOCUWARE_CHURN/only_ts_df"+str(id_)+".csv", file_format_type="csv", format_type_options={"COMPRESSION": "NONE"}, header=True, overwrite=True )
        
        only_ts_df = sum_df
        df_test = session.create_dataframe(sum_df)
        #df_test.write.copy_into_location("@PS_DOCUWARE_CHURN/TSFRESH/all_feature_TSFRESH/sum_ts_df"+str(id_)+".csv", file_format_type="csv", format_type_options={"COMPRESSION": "NONE"}, header=True, overwrite=True )
        #print("1", only_ts_df.shape)
        
        calculated['DAYS_REMAINING'] = only_ts_df['YYYYWK'].apply(get_days_diff, args=(only_ts_df.loc[only_ts_df.index[only_ts_df.shape[0]-1], 'YYYYWK'],))
        # print("-------------------------------")
        # print(calculated['DAYS_REMAINING'].shape)  
        # print("***********")
        # print(calculated['DAYS_REMAINING'])
        # print("#####################")
        if only_ts_df.shape[0] > 1:
            df_rolled = roll_time_series(only_ts_df, column_id="CUST_ACCOUNT_NUMBER", column_sort="YYYYWK")          # roll_time_series
        elif only_ts_df.shape[0] == 1:
            flag=True
            df_rolled = only_ts_df
            df_rolled['id'] = "("+str(only_ts_df.loc[only_ts_df.index[0],'CUST_ACCOUNT_NUMBER'])+", "+str(only_ts_df.loc[only_ts_df.index[0], 'YYYYWK'])+")"
        else:
            continue
        
        columns_to_dropped = set()

        df_rolled_ = df_rolled.loc[:,~df_rolled.columns.str.contains('^(Unnamed|CUST_|YYYYWK|id)', case=False)]
    
        df_rolled_imputed = df_rolled_
        
        df_rolled_imputed['YYYYWK'] = df_rolled['YYYYWK'].to_list()
        df_rolled_imputed['ID'] = df_rolled['id'].to_list()
        df_rolled_imputed['CUST_ACCOUNT_NUMBER'] = df_rolled['CUST_ACCOUNT_NUMBER'].to_list()
        #df_rolled_imputed.rename(columns={'USED_STORAGE__MB':'USED_STORAGE_MB'}, errors="raise",inplace=True)
        
        columns_to_dropped = set()

        df_rolled_imputed = df_rolled_imputed.loc[:,~df_rolled_imputed.columns.str.contains('^Unnamed', case=False)]
    
        extraction_settings = ComprehensiveFCParameters()

        if df_rolled_imputed.shape[0] > 0:
            features = extract_features(df_rolled_imputed, column_id='ID', column_sort='YYYYWK', default_fc_parameters=extraction_settings) #extract_features
            print("2",features.shape)
            #df_temp = (features.sort_index().iloc[features.shape[0]-1, :]).to_frame().transpose()
            #df_temp = (features.sort_index().loc[:, :]).transpose()
            #print(calculated['DAYS_REMAINING'])
            features['CUST_ACCOUNT_NUMBER'] = id_
            features = features.reset_index(drop=True)
            print("3",features.shape)
            
            #features_df = session.create_dataframe(features)
            #features_df.write.copy_into_location("@PS_DOCUWARE_CHURN/TSFRESH/all_feature_TSFRESH/features_df"+str(id_)+".csv", file_format_type="csv", format_type_options={"COMPRESSION": "NONE"}, header=True, overwrite=True )
        
        
            #features_with_age = pd.concat([features, calculated], axis=1, ignore_index=True)

            #features_with_age_df = session.create_dataframe(features_with_age)
            #features_with_age_df.write.copy_into_location("@PS_DOCUWARE_CHURN/TSFRESH/all_feature_TSFRESH/features_with_age_df"+str(id_)+".csv", file_format_type="csv", format_type_options={"COMPRESSION": "NONE"}, header=True, overwrite=True )

            print("4",features.shape)
            final_features = pd.concat([final_features, features], axis=0, ignore_index=True)
            final_calculated = pd.concat([final_calculated, calculated], axis=0, ignore_index=True)
            #print(calculated['DAYS_REMAINING'])
            
            #final_features_df = session.create_dataframe(final_features)
            #final_features_df.write.copy_into_location("@PS_DOCUWARE_CHURN/TSFRESH/all_feature_TSFRESH/final_features_df"+str(id_)+".csv", file_format_type="csv", format_type_options={"COMPRESSION": "NONE"}, header=True, overwrite=True )

            print("5", final_features.shape)
            print("6", final_calculated.shape)
            
        del only_ts_df
        del features
        del df_rolled
        del df_rolled_
        del df_rolled_imputed
        del calculated
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        #print(f"for Cust ID = {id_} Elapsed time: {elapsed_time} seconds")
        count+=1
        #if count==5:
        #    break
    end_time_begin = time.time()
    elapsed_time_begin = end_time_begin - start_time_begin
    #print(f"Total Elapsed time: {elapsed_time_begin} seconds")

    #Remove unwanted columns
    final_features_columns = final_features.columns
    columns_to_be_dropped_from_ts_df = []
    #print(final_features.columns)
    for col in final_features_columns:
        if re.search(r"^(Unnamed:|obr_|prtar|CONTRACT_LINE_ITEMS|PROBABILITY_OF_DELINQUENCY|RICOH_CUSTOM_RISK_MODEL|CUST_ACCOUNT_NUMBER_)", col):
            columns_to_be_dropped_from_ts_df.append(col)

    final_features_filtered = final_features.drop(columns_to_be_dropped_from_ts_df, axis=1)
    
    return final_features_filtered, final_calculated

In [ ]:
#ts_comprehensive_df, ts_age_df = engineer_timeseries_cols_using_tsfresh(final_features_for_FE)
#ts_comprehensive_df_sf = session.create_dataframe(ts_comprehensive_df)
#ts_comprehensive_df_sf.write.copy_into_location("@PS_DOCUWARE_CHURN/TSFRESH/all_feature_TSFRESH/ts_comprehensive_df.csv", file_format_type="csv", format_type_options={"COMPRESSION": "NONE"}, header=True, overwrite=True )
        
#session.write_pandas(ts_comprehensive_df, "PS_DOCUWARE_TSFRESH_FOR_TRAINING", auto_create_table=True, overwrite = True)
#session.write_pandas(ts_age_df, "PS_DOCUWARE_TSFRESH_AGE", auto_create_table=True, overwrite = True)

In [ ]:
#ts_comprehensive_df.shape, ts_age_df.shape

In [ ]:
#ts_age_df.head()

# UTILITY Function 6

In [ ]:
def global_shap_importance(model, X):
    """ Return a dataframe containing the features sorted by Shap importance
    Parameters
    ----------
    model : The tree-based model 
    X : pd.Dataframe
         training set/test set/the whole dataset ... (without the label)
    Returns
    -------
    pd.Dataframe
        A dataframe containing the features sorted by Shap importance
    """
    explainer = shap.Explainer(model)
    shap_values = explainer(X)
    cohorts = {"": shap_values}
    cohort_labels = list(cohorts.keys())
    cohort_exps = list(cohorts.values())
    for i in range(len(cohort_exps)):
        if len(cohort_exps[i].shape) == 2:
            cohort_exps[i] = cohort_exps[i].abs.mean(0)
    features = cohort_exps[0].data
    feature_names = cohort_exps[0].feature_names
    values = np.array([cohort_exps[i].values for i in range(len(cohort_exps))])
    feature_importance = pd.DataFrame(
        list(zip(feature_names, sum(values))), columns=['features', 'importance'])
    feature_importance.sort_values(
        by=['importance'], ascending=False, inplace=True)
    return feature_importance

# Raw Data Extraction

In [ ]:
CREATE OR REPLACE VIEW RUS_AIML.PS_DOCUWARE_PAYMENTS_V AS SELECT CUSTOMER_NO, RECEIPT_DATE, FUNCTIONAL_AMOUNT FROM RUS_AIML.PS_DOCUWARE_PAYMENTS;

CREATE OR REPLACE VIEW RUS_AIML.PS_DOCUWARE_REVENUE_V AS SELECT CUST_ACCOUNT_NUMBER, DATE_INVOICE_GL_DATE, INVOICE_REVLINE_TOTAL FROM RUS_AIML.PS_DOCUWARE_REVENUE;

CREATE OR REPLACE VIEW RUS_AIML.PS_DOCUWARE_TRX_V AS SELECT ACCOUNT_NUMBER, TRX_DATE, ORIGINAL_AMOUNT_DUE FROM RUS_AIML.PS_DOCUWARE_TRX;

CREATE OR REPLACE VIEW RUS_AIML.PS_DOCUWARE_CONTRACTS_SUBLINE_V AS SELECT CUST_ACCOUNT_NUMBER, SLINE_START_DATE, SLINE_END_DATE, SLINE_STATUS FROM RUS_AIML.PS_DOCUWARE_CONTRACT_SUBLINE;

CREATE OR REPLACE VIEW RUS_AIML.PS_DOCUWARE_RENEWALS_V AS SELECT TO_CHAR(BILLTOCUSTOMERNUMBER) AS BILLTOCUSTOMERNUMBER, TO_CHAR(SHIPTOCUSTNUM) AS SHIPTOCUSTNUM, STARTDATECOVERAGE, CONTRACT_END_DATE FROM RUS_AIML.PS_DOCUWARE_SSCD_RENEWALS;

CREATE OR REPLACE VIEW RUS_AIML.PS_DOCUWARE_L1_CUST_V AS SELECT CUST_ACCOUNT_NUMBER, CUST_PARTY_NAME, L3_RISE_CONSOLIDATED_NUMBER, L3_RISE_CONSOLIDATED_NAME, L2_RISE_CONSOLIDATED_NUMBER, L2_RISE_CONSOLIDATED_NAME, CUST_ACCOUNT_TYPE, CUSTOMER_SEGMENT, CUSTOMER_SEGMENT_LEVEL, CHURNED_FLAG, CHURN_DATE FROM RUS_AIML.PS_DOCUWARE_L1_CUST;

CREATE OR REPLACE VIEW RUS_AIML.DNB_RISK_BREAKDOWN_V AS SELECT TO_CHAR(ACCOUNT_NUMBER) AS ACCOUNT_NUMBER, OVERALL_BUSINESS_RISK, RICOH_CUSTOM_RISK_MODEL, PROBABILITY_OF_DELINQUENCY, PAYMENT_RISK_TRIPLE_A_RATING FROM RUS_AIML.DNB_RISK_BREAKDOWN;

CREATE OR REPLACE VIEW RUS_AIML.DOCUWARE_USAGE_JAPAN_V1_LATEST_V AS
 WITH latback AS (
   SELECT DISTINCT CONTRACT_NUMBER
    ,REGEXP_REPLACE(trim(CUSTOMER_NAME), '  ', ' ') AS CUSTOMER_NAME
    ,CONTRACT_END
    ,DOCUMENTS_OPENED
    ,USED_STORAGE__MB
    ,CONTRACT_LINE_ITEMS
    ,PERIOD
    ,YYYYWK
   FROM DOCUWARE_SHARED__REP_EUC1.PUBLIC.DOCUWARE_USAGE_JAPAN_V1_LATEST
   WHERE RICOH_REGIONS = 'US/Canada'
    AND ADP__CURRENT != 'Ricoh Canada Inc.'
   
   UNION ALL
   
   SELECT DISTINCT CONTRACT_NUMBER
    ,REGEXP_REPLACE(trim(CUSTOMER_NAME), '  ', ' ') AS CUSTOMER_NAME
    ,CONTRACT_END
    ,DOCUMENTS_OPENED
    ,USED_STORAGE__MB
    ,CONTRACT_LINE_ITEMS
    ,PERIOD
    ,YYYYWK
   FROM DOCUWARE_SHARED__REP_EUC1.PUBLIC.DOCUWARE_USAGE_JAPAN_V1_BACKUP
   WHERE RICOH_REGIONS = 'US/Canada'
    AND ADP__CURRENT != 'Ricoh Canada Inc.'
   )
  ,jaro AS (
   SELECT DISTINCT a.CUST_ACCOUNT_NUMBER
    ,b.CONTRACT_NUMBER
    ,a.CUST_PARTY_NAME
    ,b.CUSTOMER_NAME
    ,b.CONTRACT_END
    ,b.DOCUMENTS_OPENED
    ,b.USED_STORAGE__MB
    ,b.CONTRACT_LINE_ITEMS
    ,b.PERIOD
    ,b.YYYYWK
    ,jarowinkler_similarity(a.CUST_PARTY_NAME, b.CUSTOMER_NAME)
    ,rank() OVER (
     PARTITION BY a.CUST_ACCOUNT_NUMBER ORDER BY jarowinkler_similarity(a.CUST_PARTY_NAME, b.CUSTOMER_NAME) DESC
     ) AS match_rank
   FROM RUS_AIML.PS_DOCUWARE_L1_CUST a
   FULL OUTER JOIN latback b
   WHERE jarowinkler_similarity(a.CUST_PARTY_NAME, b.CUSTOMER_NAME) BETWEEN 93
     AND 100
   )
SELECT *
FROM jaro
WHERE jaro.match_rank = 1
 AND EXISTS (
  SELECT 1
  FROM jaro
  );
 

# Preprocessing Raw Data

In [ ]:
#merge_final

In [ ]:
#l1_cust_df.head()

In [ ]:
-- SELECT * FROM RUS_AIML.PS_DOCUWARE_L1_CUST limit 2;

In [ ]:
# #Fetching from previous SPROC output table
# raw_df = session.sql("SELECT * FROM RAC_RAPID_DEV.RUS_AIML.PS_DOCUWARE_RAW_DATA_EXTRACTION").to_pandas()

# non_ts_numeric_cols = ["PROBABILITY_OF_DELINQUENCY", "RICOH_CUSTOM_RISK_MODEL"]
# non_ts_categorical_cols = ["OVERALL_BUSINESS_RISK", "PAYMENT_RISK_TRIPLE_A_RATING", "CONTRACT_LINE_ITEMS"]
# columns_to_be_processed_later = non_ts_numeric_cols + non_ts_categorical_cols + ["CUST_ACCOUNT_NUMBER", "LIFESPAN_MONTHS"]
# finalized_df_ohe_to_process = raw_df.groupby("CUST_ACCOUNT_NUMBER")[columns_to_be_processed_later].first()

# # Imputation for Non Time Series columns
# pofd_median = finalized_df_ohe_to_process["PROBABILITY_OF_DELINQUENCY"].median()
# finalized_df_ohe_to_process["PROBABILITY_OF_DELINQUENCY"] = finalized_df_ohe_to_process["PROBABILITY_OF_DELINQUENCY"].apply(lambda x:   float(pofd_median) if np.isnan(x) else x)

# temp=finalized_df_ohe_to_process["RICOH_CUSTOM_RISK_MODEL"].mode().to_frame()
# rcrm_mode = temp.loc[temp.index[0], 'RICOH_CUSTOM_RISK_MODEL']
# finalized_df_ohe_to_process["RICOH_CUSTOM_RISK_MODEL"] = finalized_df_ohe_to_process["RICOH_CUSTOM_RISK_MODEL"].apply(lambda x: float(rcrm_mode) if np.isnan(x) else x)



# Data Imputation and Feature Engineering to make Training Set

In [ ]:
#try:
#Fetching from previous SPROC output table
raw_df = session.sql("SELECT * FROM RAC_RAPID_DEV.RUS_AIML.PS_DOCUWARE_RAW_DATA_EXTRACTION").to_pandas()

non_ts_numeric_cols = ["PROBABILITY_OF_DELINQUENCY", "RICOH_CUSTOM_RISK_MODEL"]
non_ts_categorical_cols = ["OVERALL_BUSINESS_RISK", "PAYMENT_RISK_TRIPLE_A_RATING", "CONTRACT_LINE_ITEMS"]
columns_to_be_processed_later = non_ts_numeric_cols + non_ts_categorical_cols + ["CUST_ACCOUNT_NUMBER", "LIFESPAN_MONTHS", "DAYS_TO_CHURN"]
finalized_df_ohe_to_process = raw_df.groupby("CUST_ACCOUNT_NUMBER")[columns_to_be_processed_later].first()

# Imputation for Non Time Series columns
pofd_median = finalized_df_ohe_to_process["PROBABILITY_OF_DELINQUENCY"].median()
finalized_df_ohe_to_process["PROBABILITY_OF_DELINQUENCY"] = finalized_df_ohe_to_process["PROBABILITY_OF_DELINQUENCY"].apply(lambda x:   float(pofd_median) if np.isnan(x) else x)

#rcrm_mode = finalized_df_ohe_to_process["RICOH_CUSTOM_RISK_MODEL"].mode()
temp=finalized_df_ohe_to_process["RICOH_CUSTOM_RISK_MODEL"].mode().to_frame()
rcrm_mode = temp.loc[temp.index[0], 'RICOH_CUSTOM_RISK_MODEL']
finalized_df_ohe_to_process["RICOH_CUSTOM_RISK_MODEL"] = finalized_df_ohe_to_process["RICOH_CUSTOM_RISK_MODEL"].apply(lambda x: float(rcrm_mode) if np.isnan(x) else x)

# One Hot Encoding for Categorical variables OVERALL_BUSINESS_RISK and PAYMENT_RISK_TRIPLE_A_RATING
finalized_df_ohe_to_process = finalized_df_ohe_to_process.reset_index(drop=True)

categorical_cols = ["OVERALL_BUSINESS_RISK", "PAYMENT_RISK_TRIPLE_A_RATING"]
fill_value = "UNK"

for col in categorical_cols:
    finalized_df_ohe_to_process[col].fillna(fill_value, inplace=True)
    finalized_df_ohe_to_process[col] = finalized_df_ohe_to_process[col].str.replace(" ", "_", regex=False)
    
    if col == "OVERALL_BUSINESS_RISK":
        col_abreviation = "obr_"
    else:
        col_abreviation = "prtar_"
        
    le_ohe = LabelEncoder()
    ohe = OneHotEncoder(handle_unknown = "ignore")
    enc_train = le_ohe.fit_transform(finalized_df_ohe_to_process[col]).reshape(finalized_df_ohe_to_process.shape[0],1)
    ohe_train = ohe.fit_transform(enc_train)
    le_ohe_name_mapping = dict(zip(le_ohe.classes_, le_ohe.transform(le_ohe.classes_)))
    
    enc_train = finalized_df_ohe_to_process[col].map(le_ohe_name_mapping).ravel().reshape(-1,1)
    enc_train[np.isnan(enc_train)] = 9999

    cols = [col_abreviation + str(x) for x in le_ohe_name_mapping.keys()]
    finalized_df_ohe_to_process = pd.concat([finalized_df_ohe_to_process.reset_index(), pd.DataFrame.sparse.from_spmatrix(ohe_train, columns = cols)], axis = 1).drop(["index"], axis=1)
    finalized_df_ohe_to_process.drop([col], axis = 1, inplace=True)

columns_to_be_droped = non_ts_categorical_cols+non_ts_numeric_cols
raw_df.drop(columns_to_be_droped, axis=1, inplace=True)

# Target Encoding for CONTRACT_LINE_ITEMS
finalized_df_ohe_to_process["CONTRACT_LINE_ITEMS"].fillna("NA", inplace=True)
finalized_df_ohe_to_process["CONTRACT_LINE_ITEMS"] = finalized_df_ohe_to_process["CONTRACT_LINE_ITEMS"].str.replace(r"\\d+x ", "", regex=True)

for i, row in finalized_df_ohe_to_process.iterrows():
    t = row["CONTRACT_LINE_ITEMS"]
    arr = t.split("-")
    arr = [x.strip() for x in arr]
    arr_s = sorted(arr)
    key = "-".join([s for s in arr_s])
    finalized_df_ohe_to_process.loc[i, "CONTRACT_LINE_ITEMS"] = key

X_train = finalized_df_ohe_to_process.copy()
y_train = finalized_df_ohe_to_process["LIFESPAN_MONTHS"]

enc = TargetEncoder(cols = ["CONTRACT_LINE_ITEMS"]).fit(X_train, y_train)
X_train_encoded = enc.transform(X_train)

now = datetime.now()
date_string = now.strftime("%m-%d-%Y")

#encoder_filename = "@RAC_RAPID_DEV.RUS_AIML.PS_DOCUWARE_CHURN/ps_docuware_target_encoder_object/ENC_CURRENT.joblib"
#encoder_filename_ar = "@RAC_RAPID_DEV.RUS_AIML.PS_DOCUWARE_CHURN/ps_docuware_target_encoder_object/archive/ENC_"+date_string+".joblib"

#joblib.dump(enc,  encoder_filename)
#joblib.dump(enc, encoder_filename_ar)

import_dir = sys._xoptions.get("snowflake_import_directory")
model_file = os.path.join("/tmp", "ENC_CURRENT.joblib.gz")
dump(enc, model_file)
#session.file.put(model_file, "@PS_DOCUWARE_CHURN",overwrite=True)
session.file.put(model_file, "@PS_DOCUWARE_CHURN/ps_docuware_target_encoder_object",overwrite=True)

model_file_ar = os.path.join("/tmp", "ENC_"+date_string+".joblib.gz")
dump(enc, model_file_ar)
session.file.put(model_file_ar, "@PS_DOCUWARE_CHURN/ps_docuware_target_encoder_object/archive",overwrite=True)

# Imputation for Time Series columns
ts_columns = ["CUST_ACCOUNT_NUMBER", "YYYYWK", "DOCUMENTS_OPENED", "USED_STORAGE__MB", "INVOICE_REVLINE_TOTAL", "ORIGINAL_AMOUNT_DUE", "FUNCTIONAL_AMOUNT"]
raw_df["transformed_YYYYWK"] = raw_df["MONTH"].apply(convert_date_to_yyyywk)
#Impute missing YYYYWK with equivalent MONTH
raw_df["YYYYWK"].fillna(raw_df["transformed_YYYYWK"], inplace=True)
raw_df.drop("transformed_YYYYWK", axis=1, inplace=True)
ts_df = raw_df[ts_columns]
ts_df = ts_df[ts_df['YYYYWK'].notna()]
ts_df['YYYYWK'] = ts_df['YYYYWK'].astype(int)
ts_df['CUST_ACCOUNT_NUMBER'] = ts_df['CUST_ACCOUNT_NUMBER'].astype(int)
ts_df.rename(columns={'USED_STORAGE__MB':'USED_STORAGE_MB'},inplace=True)
ts_df_sorted = ts_df.sort_values(['CUST_ACCOUNT_NUMBER','YYYYWK']).drop_duplicates()

only_features = ts_df_sorted.copy()
only_features = only_features.fillna(0)



In [ ]:
only_features.head()

In [ ]:
session.write_pandas(only_features, "PS_DOCUWARE_RAW_TS_FEATURES", auto_create_table=True, overwrite = True)

In [ ]:
lifespan_df = raw_df[['CUST_ACCOUNT_NUMBER', 'DAYS_TO_CHURN']]
lifespan_df['CUST_ACCOUNT_NUMBER'] = lifespan_df['CUST_ACCOUNT_NUMBER'].astype(int)
lifespan_df.drop_duplicates(inplace=True)
lifespan_df.shape

In [ ]:
"""df_churn_cust = l1_cust_churned[["CUST_ACCOUNT_NUMBER", "CHURN_DATE"]]
df_churn_cust['CHURN_YYYYWK_DATE'] = df_churn_cust['CHURN_DATE'].apply(convert_date_to_yyyywk)
df_churn_cust['CUST_ACCOUNT_NUMBER'] = df_churn_cust['CUST_ACCOUNT_NUMBER'].astype(int)
#print(only_features.shape) # (14614, 7)
df_raw_churn = only_features.merge(df_churn_cust, on='CUST_ACCOUNT_NUMBER', how='inner')
#print(df_raw_churn.shape) # (14614, 9)
final_features = df_raw_churn[df_raw_churn['YYYYWK'] <= df_raw_churn['CHURN_YYYYWK_DATE']]
print(final_features.shape)"""  #commented out CK 5-19

In [ ]:
final_features.head()

In [ ]:
final_features_for_FE = final_features.drop(['CHURN_DATE', 'CHURN_YYYYWK_DATE'], axis=1)
#print(final_features_for_FE.shape) # (5551, 7)
final_features_for_FE = final_features_for_FE.merge(lifespan_df, on='CUST_ACCOUNT_NUMBER', how='inner')
#print(final_features_for_FE.shape) # (5551, 8)
final_features_for_FE.head()

In [ ]:
final_features_for_FE.head(5551)

In [ ]:
print(X_train_encoded['CONTRACT_LINE_ITEMS'])

# Store time series and Non time series data into DB before feature engineering from TSFRESH

In [ ]:
session.write_pandas(X_train_encoded, "PS_DOCUWARE_RAW_X_TRAIN_ENCODED_DATA_BEFORE_TSFRESH", auto_create_table=True, overwrite = True)
session.write_pandas(final_features_for_FE, "PS_DOCUWARE_RAW_DATA_BEFORE_TSFRESH", auto_create_table=True, overwrite = True)

In [ ]:
#Fetching all the different views
pay_df = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_PAYMENTS_V").to_pandas()
rev_df = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_REVENUE_V").to_pandas()
trx_df = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_TRX_V").to_pandas()
contracts_sub_df = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_CONTRACTS_SUBLINE_V").to_pandas()
renewals_df = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_RENEWALS_V").to_pandas()
l1_cust_df =  session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_L1_CUST_V").to_pandas()
dnb_risk_df = session.sql("SELECT * FROM RUS_AIML.DNB_RISK_BREAKDOWN_V").to_pandas()
usage_latest = session.sql("SELECT * FROM RUS_AIML.DOCUWARE_USAGE_JAPAN_V1_LATEST_V").to_pandas()

# Removing duplicates from all the pandas dataframes
pay_df = pay_df.drop_duplicates()
rev_df = rev_df.drop_duplicates()
trx_df = trx_df.drop_duplicates()
contracts_sub_df = contracts_sub_df.drop_duplicates()
renewals_df = renewals_df.drop_duplicates()
l1_cust_df = l1_cust_df.drop_duplicates()
dnb_risk_df = dnb_risk_df.drop_duplicates()
usage_latest = usage_latest.drop_duplicates()

# Selecting only churned customers

l1_cust_churned = l1_cust_df[l1_cust_df["CHURNED_FLAG"]==True]

pay_df_churned = pay_df.merge(l1_cust_churned[["CUST_ACCOUNT_NUMBER"]], left_on = "CUSTOMER_NO", right_on = "CUST_ACCOUNT_NUMBER", how="inner")
pay_df_churned = pay_df_churned.drop("CUSTOMER_NO", axis=1)
pay_df_churned["MONTH"] = pd.to_datetime(pay_df_churned["RECEIPT_DATE"]).dt.to_period("M").dt.to_timestamp()

rev_df_churned = rev_df.merge(l1_cust_churned[["CUST_ACCOUNT_NUMBER"]], on = "CUST_ACCOUNT_NUMBER", how="inner")
rev_df_churned["MONTH"] = pd.to_datetime(rev_df_churned["DATE_INVOICE_GL_DATE"]).dt.to_period("M").dt.to_timestamp()

trx_df_churned = trx_df.merge(l1_cust_churned[["CUST_ACCOUNT_NUMBER"]], left_on = "ACCOUNT_NUMBER", right_on = "CUST_ACCOUNT_NUMBER", how="inner")
trx_df_churned = trx_df_churned.drop("ACCOUNT_NUMBER", axis=1)
trx_df_churned["MONTH"] = pd.to_datetime(trx_df_churned["TRX_DATE"]).dt.to_period("M").dt.to_timestamp()

p_r_t_merged = pay_df_churned.merge(rev_df_churned, on = ["CUST_ACCOUNT_NUMBER","MONTH"], how="outer").merge(trx_df_churned, on = ["CUST_ACCOUNT_NUMBER","MONTH"], how="outer")

#Contracts Subline
contracts_sub_df["SLINE_START_DATE"] = pd.to_datetime(contracts_sub_df["SLINE_START_DATE"])
contracts_sub_df["SLINE_END_DATE"] = pd.to_datetime(contracts_sub_df["SLINE_END_DATE"])
contracts_sub_df["SUB_START_MONTH"] = pd.to_datetime(contracts_sub_df["SLINE_START_DATE"]).dt.to_period("M").dt.to_timestamp()
contracts_sub_df["SUB_END_MONTH"] = pd.to_datetime(contracts_sub_df["SLINE_END_DATE"]).dt.to_period("M").dt.to_timestamp()
contracts_sub_df["SUB_EARLIEST_MONTH"] = contracts_sub_df.groupby("CUST_ACCOUNT_NUMBER")["SUB_START_MONTH"].transform("min")
contracts_sub_df["SUB_LATEST_MONTH"] = contracts_sub_df.groupby("CUST_ACCOUNT_NUMBER")["SUB_END_MONTH"].transform("max")
contracts_sub_df_churned = contracts_sub_df.merge(l1_cust_churned[["CUST_ACCOUNT_NUMBER"]], on = "CUST_ACCOUNT_NUMBER", how="inner")

contracts_sub_df_churned = contracts_sub_df_churned.drop_duplicates()

# Renewals
cols_to_str = ["BILLTOCUSTOMERNUMBER","SHIPTOCUSTNUM"]
renewals_df[cols_to_str] = renewals_df[cols_to_str].astype("Int64").astype(str)

renewals_df_churned_1 = renewals_df.merge(l1_cust_churned[["CUST_ACCOUNT_NUMBER"]], left_on = "SHIPTOCUSTNUM", right_on = "CUST_ACCOUNT_NUMBER",  how="left")

renewals_df_churned_2 = renewals_df_churned_1.merge(l1_cust_churned[["CUST_ACCOUNT_NUMBER"]], left_on = "BILLTOCUSTOMERNUMBER", right_on = "CUST_ACCOUNT_NUMBER",  how="left", suffixes = ("_1", "_2"))
renewals_df_churned_2["CUST_ACCOUNT_NUMBER"] = renewals_df_churned_2["CUST_ACCOUNT_NUMBER_1"].fillna(renewals_df_churned_2["CUST_ACCOUNT_NUMBER_2"])
renewals_df_churned_2 = renewals_df_churned_2.drop(["BILLTOCUSTOMERNUMBER", "SHIPTOCUSTNUM","CUST_ACCOUNT_NUMBER_1","CUST_ACCOUNT_NUMBER_2"], axis=1)
renewals_df_churned_2 = renewals_df_churned_2.dropna()

renewals_df_churned_2["STARTDATECOVERAGE"] =  pd.to_datetime(renewals_df_churned_2["STARTDATECOVERAGE"])
renewals_df_churned_2["RENEWALS_START_MONTH"] = pd.to_datetime(renewals_df_churned_2["STARTDATECOVERAGE"]).dt.to_period("M").dt.to_timestamp()
renewals_df_churned_2["RENEWALS_END_MONTH"] = pd.to_datetime(renewals_df_churned_2["CONTRACT_END_DATE"]).dt.to_period("M").dt.to_timestamp()

renewals_df_churned_2["RENEWALS_EARLIEST_MONTH"] = renewals_df_churned_2.groupby(renewals_df_churned_2["CUST_ACCOUNT_NUMBER"])["RENEWALS_START_MONTH"].transform("min")
renewals_df_churned_2["RENEWALS_LATEST_MONTH"] = renewals_df_churned_2.groupby(renewals_df_churned_2["CUST_ACCOUNT_NUMBER"])["RENEWALS_END_MONTH"].transform("max")

#DNB Risk Breakdown
dnb_risk_df["ACCOUNT_NUMBER"] = dnb_risk_df["ACCOUNT_NUMBER"].astype("Int64").astype(str)
dnb_risk_df_churned = dnb_risk_df.merge(l1_cust_churned[["CUST_ACCOUNT_NUMBER"]], left_on = "ACCOUNT_NUMBER", right_on = "CUST_ACCOUNT_NUMBER",  how="inner")
dnb_risk_df_churned = dnb_risk_df_churned.drop("ACCOUNT_NUMBER", axis=1)

usage_latest["YYYYWK_Transformed"] = pd.to_datetime(usage_latest["YYYYWK"].apply(convert_yyyywk_to_date), errors = "coerce")

#Usage Japan Latest
usage_latest["CUST_ACCOUNT_NUMBER"] = usage_latest["CUST_ACCOUNT_NUMBER"].astype("Int64").astype(str)
usage_latest["YYYYWK_MONTH"] = pd.to_datetime(usage_latest["YYYYWK_Transformed"]).dt.to_period("M").dt.to_timestamp()
usage_latest_churned = usage_latest.merge(l1_cust_churned[["CUST_ACCOUNT_NUMBER"]], on="CUST_ACCOUNT_NUMBER",  how="inner")

# Merging all the churned customers data frames i.e. Payments, Revenue, Transactions, contracts, contracts subline, contracts topline, renewals, snow inc, tech survey, loyalty survey, dnb risk and usage latest

merged_1 = p_r_t_merged.merge(contracts_sub_df_churned, left_on = ["CUST_ACCOUNT_NUMBER", "MONTH"], right_on = ["CUST_ACCOUNT_NUMBER", "SUB_START_MONTH"], how="outer")
merged_1["MONTH"] = merged_1["MONTH"].fillna(merged_1["SUB_START_MONTH"])
merged_1 = merged_1.drop("SUB_START_MONTH", axis=1)

merged_2 = merged_1.merge(renewals_df_churned_2, left_on = ["CUST_ACCOUNT_NUMBER","MONTH"], right_on = ["CUST_ACCOUNT_NUMBER","RENEWALS_START_MONTH"], how="outer")
merged_2["MONTH"] = merged_2["MONTH"].fillna(merged_2["RENEWALS_START_MONTH"])
merged_2 = merged_2.drop("RENEWALS_START_MONTH", axis=1)

merged_3 = merged_2.merge(dnb_risk_df_churned, on="CUST_ACCOUNT_NUMBER", how="left")

merged_4 = merged_3.merge(usage_latest_churned, left_on= ["CUST_ACCOUNT_NUMBER","MONTH"], right_on = ["CUST_ACCOUNT_NUMBER", "YYYYWK_MONTH"], how="outer")
merged_4["MONTH"] = merged_4["MONTH"].fillna(merged_4["YYYYWK_MONTH"])

to_drop_2 = ["CONTRACT_NUMBER", "CUST_PARTY_NAME", "CUSTOMER_NAME", "CONTRACT_END","JAROWINKLER_SIMILARITY(A.CUST_PARTY_NAME, B.CUSTOMER_NAME)"]

merged_4 = merged_4.drop(to_drop_2, axis=1)

merged_5 = merged_4.merge(l1_cust_churned[["CUST_ACCOUNT_NUMBER", "CHURNED_FLAG", "CHURN_DATE"]], on = "CUST_ACCOUNT_NUMBER", how="inner")

merged_5["EARLIEST_DATE"] = merged_5[["RECEIPT_DATE", "DATE_INVOICE_GL_DATE", "TRX_DATE", "SLINE_START_DATE", "STARTDATECOVERAGE"]].min(axis=1)
merged_5["FINAL_EARLIEST_DATE"] = merged_5.groupby("CUST_ACCOUNT_NUMBER")["EARLIEST_DATE"].transform("min")

merged_5["CHURN_DATE"] = pd.to_datetime(merged_5["CHURN_DATE"])
merged_5["CHURN_MONTH"] = pd.to_datetime(merged_5["CHURN_DATE"]).dt.to_period("M").dt.to_timestamp()


merged_5["LIFESPAN_MONTHS"] = ((merged_5["CHURN_DATE"] - merged_5["FINAL_EARLIEST_DATE"]).dt.days) / 30
merged_5["DAYS_TO_CHURN"]  = ((merged_5["CHURN_DATE"] - merged_5["FINAL_EARLIEST_DATE"]).dt.days)

to_drop_3 = ["RECEIPT_DATE", "DATE_INVOICE_GL_DATE", "TRX_DATE", "SLINE_START_DATE", "SLINE_END_DATE", "SUB_END_MONTH", "SLINE_STATUS", "SUB_EARLIEST_MONTH", "SUB_LATEST_MONTH", "STARTDATECOVERAGE", "CONTRACT_END_DATE", "RENEWALS_END_MONTH", "RENEWALS_EARLIEST_MONTH","RENEWALS_LATEST_MONTH", "YYYYWK_MONTH", "PERIOD", "CHURNED_FLAG","CHURN_MONTH", "EARLIEST_DATE"]

merged_5 = merged_5.drop(to_drop_3, axis=1)


date_col = ["MONTH","YYYYWK_Transformed", "FINAL_EARLIEST_DATE", "CHURN_DATE"]

merged_5[date_col] = merged_5[date_col].astype(str)
merged_5 = merged_5.drop_duplicates()

usage_latest_churned['CUST_ACCOUNT_NUMBER'].unique()

temp = pd.DataFrame({'CUST_ACCOUNT_NUMBER':usage_latest_churned['CUST_ACCOUNT_NUMBER'].unique()})
merge_final = merged_5.merge(temp, on='CUST_ACCOUNT_NUMBER', how='inner')

session.write_pandas(merge_final, "PS_DOCUWARE_RAW_DATA_EXTRACTION", auto_create_table=True, overwrite = True)


# Starting TSFRESH API

In [ ]:
#ts_comprehensive_df = time_series_ts_fresh_features(only_features)
ts_comprehensive_df, final_ts_age  = engineer_timeseries_cols_using_tsfresh(final_features_for_FE)

#ts_comprehensive_df = session.sql("SELECT * FROM RAC_RAPID_DEV.RUS_AIML.PS_DOCUWARE_ONLY_TS_DF").to_pandas()

    
#except Exception as e: 
#    print("FAIL(Post Processing)!" + " Error: " + str(e))
ts_comprehensive_df = ts_comprehensive_df.reset_index(drop=True)
final_ts_age = final_ts_age.reset_index(drop=True)

final_ts_df = pd.concat([ts_comprehensive_df, final_ts_age], axis=1, ignore_index=True)
#session.write_pandas(final_ts_df, "PS_DOCUWARE_TSFRESH_FEATURES_WITH_AGE", auto_create_table=True, overwrite = True)

all_columns = ts_comprehensive_df.columns.tolist()+final_ts_age.columns.tolist()
final_ts_df.columns = all_columns
final_ts_df.head()

In [ ]:
final_ts_age.head()

In [ ]:
for col in final_ts_df.columns:
    if re.search("^DAYS", col):
        print(col)

In [ ]:
ts_comprehensive_df = final_ts_df.copy()
ts_comprehensive_df.shape

In [ ]:
# Check if it contains CUST_ACCOUNT_NUMBER related TSFRESH features

for col in ts_comprehensive_df.columns:
    if re.search("^CUST_ACCOUNT_NUMBER_", col):
        print(col)
    if re.search("^DAYS", col):
        print(col)

In [ ]:
# Check first two words of all columns generated by TSFRESH
prefix=set()

for col in ts_comprehensive_df.columns:
    prefix.add((col.split('_')[0], col.split('_')[1]))

In [ ]:
print(prefix)

# Storing Training Data after Feature Engineering and before Training

In [ ]:
ts_comprehensive_df["CUST_ACCOUNT_NUMBER"] = ts_comprehensive_df["CUST_ACCOUNT_NUMBER"].astype(int)
X_train_encoded["CUST_ACCOUNT_NUMBER"] = X_train_encoded["CUST_ACCOUNT_NUMBER"].astype(int)
X_train_encoded = X_train_encoded.reset_index(drop=True)
ts_comprehensive_df = ts_comprehensive_df.reset_index(drop=True)
comprehensive_imputed_df = pd.merge(ts_comprehensive_df, X_train_encoded, on="CUST_ACCOUNT_NUMBER", how="inner")

In [ ]:
ts_comprehensive_df.shape, X_train_encoded.shape, comprehensive_imputed_df.shape

In [ ]:
for col in ts_comprehensive_df.columns:
    if re.search("^DAYS", col):
        print(col)

In [ ]:
for col in comprehensive_imputed_df.columns:
    if re.search("^DAYS", col):
        print(col)

In [ ]:
for col in comprehensive_imputed_df.columns:
    if pd.api.types.is_sparse(comprehensive_imputed_df[col]):
        comprehensive_imputed_df[col] = comprehensive_imputed_df[col].sparse.to_dense()

session.write_pandas(comprehensive_imputed_df, "PS_DOCUWARE_IMPUTED_DATA", auto_create_table=True, overwrite = True)
#session.write_pandas(comprehensive_imputed_df, "PS_DOCUWARE_IMPUTED_DATA_266", auto_create_table=True, overwrite = True)

print("Successfuly created table PS_DOCUWARE_IMPUTED_DATA")

In [ ]:
ts_comprehensive_df.shape


In [ ]:
comprehensive_imputed_df.shape

In [ ]:
for col in ts_comprehensive_df.columns:
    if re.search("^DAYS", col):
        print(col)

In [ ]:
for col in X_train_encoded.columns:
    if re.search("^DAYS", col):
        print(col)

In [ ]:
for col in comprehensive_imputed_df.columns:
    if re.search("^DAYS", col):
        print(col)

In [ ]:
# Check first two words of all columns generated by TSFRESH
prefix=set()

for col in comprehensive_imputed_df.columns:
    prefix.add((col.split('_')[0], col.split('_')[1]))
    prefix.add(col.split('_')[0])

In [ ]:
prefix

# Training using XGBoost Regression

In [ ]:
comprehensive_imputed_df = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_IMPUTED_DATA").to_pandas()
comprehensive_imputed_df = comprehensive_imputed_df.replace([np.inf, -np.inf], 0)
comprehensive_imputed_df = comprehensive_imputed_df.fillna(0)

for col in comprehensive_imputed_df.columns:
    comprehensive_imputed_df[col] = pd.to_numeric(comprehensive_imputed_df[col], errors="coerce").astype(float)

# comprehensive_imputed_o_df = pd.series(comprehensive_imputed_df)
# comprehensive_imputed_df = pd.to_numeric(comprehensive_imputed_o_df, errors="coerce")
# comprehensive_imputed_df = comprehensive_imputed_df.to_frame()





In [ ]:
cust_churn_df = comprehensive_imputed_df[["CUST_ACCOUNT_NUMBER","DAYS_TO_CHURN"]]
cust_churn_df.head()

In [ ]:
cust_churn_df.drop_duplicates(inplace=True)
cust_churn_df.shape

In [ ]:
cust_churn_df.head()

In [ ]:
# Stratified Sampling
stratified_df = pd.DataFrame()

label_count=1
for i in range(365, 1461, 365):
    XX = cust_churn_df[((i-365) < cust_churn_df["DAYS_TO_CHURN"]) & (cust_churn_df["DAYS_TO_CHURN"] <= i)]
    XX["SAMPLE"] = label_count
    stratified_df = pd.concat([stratified_df, XX], axis=0, ignore_index=True)
    label_count+=1
    #print("For ", (i-365)+1, " To ", i , " Total Counts = ", XX.shape[0])
    if i==1460:
        XX = cust_churn_df[1460 < cust_churn_df["DAYS_TO_CHURN"]]
        XX["SAMPLE"] = label_count
        stratified_df = pd.concat([stratified_df, XX], axis=0, ignore_index=True)
        #print("For ", 1460+1, " To upper limit Total Counts = ", (1460 < data["DAYS_TO_CHURN"]).sum())

In [ ]:
stratified_df["NO_OF_RENEWALS"] = stratified_df["SAMPLE"]-1

In [ ]:
stratified_df["SAMPLE"].value_counts()

In [ ]:
stratified_df["NO_OF_RENEWALS"].value_counts()

In [ ]:
stratified_df.head()

In [ ]:


features = stratified_df.drop(["DAYS_TO_CHURN", "SAMPLE"], axis=1)
y = stratified_df["SAMPLE"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
X_test.head()

In [ ]:
type(X_train), type(X_test), type(y_train), type(y_test)

In [ ]:
comprehensive_imputed_df.shape

In [ ]:
X_train_all_cols = pd.merge(comprehensive_imputed_df, X_train, how='inner', on='CUST_ACCOUNT_NUMBER')
X_train_all_cols.shape

In [ ]:
X_test_all_cols = pd.merge(comprehensive_imputed_df, X_test, how='inner', on='CUST_ACCOUNT_NUMBER')
X_test_all_cols.shape

In [ ]:
train_X = X_train_all_cols.drop(["CUST_ACCOUNT_NUMBER","LIFESPAN_MONTHS","DAYS_TO_CHURN", "DAYS_REMAINING"], axis=1) # Let cluster be in training
train_y = X_train_all_cols["DAYS_REMAINING"]

In [ ]:
test_X = X_test_all_cols.drop(["CUST_ACCOUNT_NUMBER","LIFESPAN_MONTHS","DAYS_TO_CHURN", "DAYS_REMAINING"], axis=1) # Let cluster be in training
test_y = X_test_all_cols["DAYS_REMAINING"]

In [ ]:
train_X.shape, train_y.shape, test_X.shape, test_y.shape

In [ ]:
type(train_X), type(test_X), type(train_y), type(test_y)

In [ ]:
# #try:
# #Fetching from previous SPROC output table
# comprehensive_imputed_df = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_IMPUTED_DATA").to_pandas()
# comprehensive_imputed_df = comprehensive_imputed_df.replace([np.inf, -np.inf], 0)
# comprehensive_imputed_df = comprehensive_imputed_df.fillna(0)

# for col in comprehensive_imputed_df.columns:
#     comprehensive_imputed_df[col] = pd.to_numeric(comprehensive_imputed_df[col], errors="coerce").astype(float)

# # comprehensive_imputed_o_df = pd.series(comprehensive_imputed_df)
# # comprehensive_imputed_df = pd.to_numeric(comprehensive_imputed_o_df, errors="coerce")
# # comprehensive_imputed_df = comprehensive_imputed_df.to_frame()

# # Stratified Sampling
# stratified_df = pd.DataFrame()

# comprehensive_imputed_df[["CUST_ACCOUNT_NUMBER", "DAYS_TO_CHURN"]]

# label_count=1
# for i in range(365, 1461, 365):
#     XX = comprehensive_imputed_df[((i-365) < comprehensive_imputed_df["DAYS_TO_CHURN"]) & (comprehensive_imputed_df["DAYS_TO_CHURN"] <= i)]
#     #XX["SAMPLE"] = label_count
#     stratified_df = pd.concat([stratified_df, XX], axis=0, ignore_index=True)
#     #label_count+=1
#     #print("For ", (i-365)+1, " To ", i , " Total Counts = ", XX.shape[0])
#     if i==1460:
#         XX = comprehensive_imputed_df[1460 < comprehensive_imputed_df["DAYS_TO_CHURN"]]
#         XX["SAMPLE"] = label_count
#         stratified_df = pd.concat([stratified_df, XX], axis=0, ignore_index=True)
#         #print("For ", 1460+1, " To upper limit Total Counts = ", (1460 < data["DAYS_TO_CHURN"]).sum())


#stratified_df["LOG_OF_MONTHS_TO_CHURN"] = np.log(stratified_df["LIFESPAN_MONTHS"])

# Stratified Sampling
#features = stratified_df.drop(["CUST_ACCOUNT_NUMBER","LIFESPAN_MONTHS","DAYS_TO_CHURN", "SAMPLE"], axis=1)
# features = comprehensive_imputed_df.drop(["CUST_ACCOUNT_NUMBER","LIFESPAN_MONTHS","DAYS_TO_CHURN", "DAYS_REMAINING"], axis=1) # Let cluster be in training
# y = comprehensive_imputed_df["DAYS_REMAINING"]

# 20/80 Split
# X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42, shuffle=True)#, stratify=y)
#y_train = X_train["LOG_OF_MONTHS_TO_CHURN"]
#y_test = X_test["LOG_OF_MONTHS_TO_CHURN"]

#X_train.drop("LOG_OF_MONTHS_TO_CHURN", axis=1, inplace=True)
#X_test.drop("LOG_OF_MONTHS_TO_CHURN", axis=1, inplace=True)

# Training

param_grid = {"learning_rate": Real(0.01, 1.0, "uniform"),
             "max_depth": Integer(2, 12),
             "subsample": Real(0.1, 1.0, "uniform"),
             "colsample_bytree": Real(0.1, 1.0, "uniform"), # subsample ratio of columns by tree
             "reg_lambda": Real(1e-9, 100., "uniform"), # L2 regularization
             "reg_alpha": Real(1e-9, 100., "uniform"), # L1 regularization
             "n_estimators": Integer(20, 5000)
}

xgb_model = xgb.XGBRegressor(tree_method="gpu_hist", random_state=10)

bayes_search = BayesSearchCV(
        estimator=xgb_model, 
        search_spaces=param_grid, 
        scoring="neg_mean_squared_error", 
        cv=5,
        n_iter=50,
        n_jobs=-1,
        verbose=3,
        random_state=0)

print("Training Starts")

bayes_search.fit(train_X, train_y)

print("Training Finishes")

best_models = bayes_search.best_estimator_

#y_pred = best_models.predict(X_test)
#rmse = np.sqrt(mean_squared_error(y_test, y_pred))

import time
import datetime
ts = time.time()
time_of_day = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d::%H:%M:%S')

print("Writing model into Staging Archive")
#import_dir = sys._xoptions.get("snowflake_import_directory")
model_file = os.path.join("/tmp", "xgb.joblib.gz")
dump(best_models, model_file)
session.file.put(model_file, "@PS_DOCUWARE_CHURN/ps_docuware_churn_model",overwrite=True)

print("Writing model into Staging")
model_file = os.path.join("/tmp", "xgb."+str(time_of_day)+".joblib.gz")
dump(best_models, model_file)
session.file.put(model_file, "@PS_DOCUWARE_CHURN/ps_docuware_churn_model/archive",overwrite=True)

print("Successfuly trained the model and stored" )   
#except Exception as e: 
#print("FAIL(Post Processing)!" + " Error: " + str(e))

# Test for churned customers

In [ ]:
comprehensive_imputed_df.loc[test_X.index,"CUST_ACCOUNT_NUMBER"].astype(str).unique()

In [ ]:
last_rows_X_test = X_test_all_cols[ X_test_all_cols["DAYS_REMAINING"] == 0]

In [ ]:
last_rows_X_test[["CUST_ACCOUNT_NUMBER", "DAYS_REMAINING"]].index

In [ ]:
test_25_X = last_rows_X_test.drop(["CUST_ACCOUNT_NUMBER","LIFESPAN_MONTHS","DAYS_TO_CHURN", "DAYS_REMAINING"], axis=1) # Let cluster be in training
test_25_y = last_rows_X_test["DAYS_REMAINING"]

In [ ]:
test_25_X.shape, test_25_y.shape

In [ ]:
test_25_y

In [ ]:
# Inferencing
y_pred = best_models.predict(test_25_X)
remaining_months = y_pred/30
print(remaining_months.shape)

actual_remaining = test_25_y/30
final_test_predicted_df = pd.DataFrame({
    "CUST_ACCOUNT_NUMBER": last_rows_X_test.loc[test_25_X.index,"CUST_ACCOUNT_NUMBER"].astype(str),
    "PREDICTED_MONTHS_REMAINING": remaining_months,
    "ACTUAL_MONTH_REMAINING": actual_remaining,
    "RESIDUAL": abs(remaining_months-actual_remaining)
})

final_test_predicted_df.head(50)

In [ ]:
# Load Training data
# comprehensive_imputed_df = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_IMPUTED_DATA").to_pandas()
# comprehensive_imputed_df = comprehensive_imputed_df.replace([np.inf, -np.inf], 0)
# comprehensive_imputed_df = comprehensive_imputed_df.fillna(0)

# for col in comprehensive_imputed_df.columns:
#     comprehensive_imputed_df[col] = pd.to_numeric(comprehensive_imputed_df[col], errors="coerce").astype(float)

# Stratified Sampling
#features = stratified_df.drop(["CUST_ACCOUNT_NUMBER","LIFESPAN_MONTHS","DAYS_TO_CHURN", "SAMPLE"], axis=1)
# features = comprehensive_imputed_df.drop(["CUST_ACCOUNT_NUMBER","LIFESPAN_MONTHS","DAYS_TO_CHURN", "DAYS_REMAINING"], axis=1) # Let cluster be in training
# y = comprehensive_imputed_df["DAYS_REMAINING"]

# # 20/80 Split
# X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42, shuffle=True)#, stratify=y)

# # Load model_file
# model_file = os.path.join('/tmp', 'xgb.joblib.gz')
# session.file.get("@PS_DOCUWARE_CHURN/ps_docuware_churn_model/xgb.joblib.gz", "/tmp")
# best_models = load(model_file)

# # Split XTrain,Ytrain


# Inferencing
y_pred = best_models.predict(test_X)
remaining_months = y_pred/30
print(remaining_months.shape)

actual_remaining = test_y/30
final_test_predicted_df = pd.DataFrame({
    "CUST_ACCOUNT_NUMBER": comprehensive_imputed_df.loc[test_X.index,"CUST_ACCOUNT_NUMBER"].astype(str),
    "PREDICTED_MONTHS_REMAINING": remaining_months,
    "ACTUAL_MONTH_REMAINING": actual_remaining,
    "RESIDUAL": abs(remaining_months-actual_remaining)
})

final_test_predicted_df.head(796)

In [ ]:
# Feature Importance Scores
importance = best_model.get_booster().get_score(importance_type='gain')

In [ ]:
importance_df = pd.DataFrame(importance.items(), columns=['Feature', 'Score'])
importance_df.head(287)

# Plot of Important Features ranked by XGBoost algorithm

In [ ]:
plt.figure(figsize=(40,30))
xgb.plot_importance(best_model, max_num_features=20)
#plt.savefig('C:/My Documents/work/Projects/Short Term Goal 2 - Customer Churn Prediction/ML Modeling Phase/Overall_Model_Importance_16Apr_2025.png')
plt.show()

# Data extrcation of Live Customers for Inferencing

In [ ]:
#Fetching all the different views
pay_df = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_PAYMENTS_V").to_pandas()
rev_df = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_REVENUE_V").to_pandas()
trx_df = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_TRX_V").to_pandas()
contracts_sub_df = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_CONTRACTS_SUBLINE_V").to_pandas()
renewals_df = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_RENEWALS_V").to_pandas()
l1_cust_df =  session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_L1_CUST_V").to_pandas()
dnb_risk_df = session.sql("SELECT * FROM RUS_AIML.DNB_RISK_BREAKDOWN_V").to_pandas()
usage_latest = session.sql("SELECT * FROM RUS_AIML.DOCUWARE_USAGE_JAPAN_V1_LATEST_V").to_pandas()

# Removing duplicates from all the pandas dataframes
pay_df = pay_df.drop_duplicates()
rev_df = rev_df.drop_duplicates()
trx_df = trx_df.drop_duplicates()
contracts_sub_df = contracts_sub_df.drop_duplicates()
renewals_df = renewals_df.drop_duplicates()
l1_cust_df = l1_cust_df.drop_duplicates()
dnb_risk_df = dnb_risk_df.drop_duplicates()
usage_latest = usage_latest.drop_duplicates()

# Selecting active customers

l1_cust_active = l1_cust_df[l1_cust_df["CHURNED_FLAG"]==False]

pay_df_active = pay_df.merge(l1_cust_active[["CUST_ACCOUNT_NUMBER"]], left_on = "CUSTOMER_NO", right_on = "CUST_ACCOUNT_NUMBER", how="inner")
pay_df_active = pay_df_active.drop("CUSTOMER_NO", axis=1)
pay_df_active["MONTH"] = pd.to_datetime(pay_df_active["RECEIPT_DATE"]).dt.to_period("M").dt.to_timestamp()

rev_df_active = rev_df.merge(l1_cust_active[["CUST_ACCOUNT_NUMBER"]], on = "CUST_ACCOUNT_NUMBER", how="inner")
rev_df_active["MONTH"] = pd.to_datetime(rev_df_active["DATE_INVOICE_GL_DATE"]).dt.to_period("M").dt.to_timestamp()

trx_df_active = trx_df.merge(l1_cust_active[["CUST_ACCOUNT_NUMBER"]], left_on = "ACCOUNT_NUMBER", right_on = "CUST_ACCOUNT_NUMBER", how="inner")
trx_df_active = trx_df_active.drop("ACCOUNT_NUMBER", axis=1)
trx_df_active["MONTH"] = pd.to_datetime(trx_df_active["TRX_DATE"]).dt.to_period("M").dt.to_timestamp()

p_r_t_merged = pay_df_active.merge(rev_df_active, on = ["CUST_ACCOUNT_NUMBER","MONTH"], how="outer").merge(trx_df_active, on = ["CUST_ACCOUNT_NUMBER","MONTH"], how="outer")

#Contracts Subline
contracts_sub_df["SLINE_START_DATE"] = pd.to_datetime(contracts_sub_df["SLINE_START_DATE"])
contracts_sub_df["SLINE_END_DATE"] = pd.to_datetime(contracts_sub_df["SLINE_END_DATE"])
contracts_sub_df["SUB_START_MONTH"] = pd.to_datetime(contracts_sub_df["SLINE_START_DATE"]).dt.to_period("M").dt.to_timestamp()
contracts_sub_df["SUB_END_MONTH"] = pd.to_datetime(contracts_sub_df["SLINE_END_DATE"]).dt.to_period("M").dt.to_timestamp()
contracts_sub_df["SUB_EARLIEST_MONTH"] = contracts_sub_df.groupby("CUST_ACCOUNT_NUMBER")["SUB_START_MONTH"].transform("min")
contracts_sub_df["SUB_LATEST_MONTH"] = contracts_sub_df.groupby("CUST_ACCOUNT_NUMBER")["SUB_END_MONTH"].transform("max")
contracts_sub_df_active = contracts_sub_df.merge(l1_cust_active[["CUST_ACCOUNT_NUMBER"]], on = "CUST_ACCOUNT_NUMBER", how="inner")

contracts_sub_df_active = contracts_sub_df_active.drop_duplicates()

# Renewals
cols_to_str = ["BILLTOCUSTOMERNUMBER","SHIPTOCUSTNUM"]
renewals_df[cols_to_str] = renewals_df[cols_to_str].astype("Int64").astype(str)

renewals_df_active_1 = renewals_df.merge(l1_cust_active[["CUST_ACCOUNT_NUMBER"]], left_on = "SHIPTOCUSTNUM", right_on = "CUST_ACCOUNT_NUMBER",  how="left")

renewals_df_active_2 = renewals_df_active_1.merge(l1_cust_active[["CUST_ACCOUNT_NUMBER"]], left_on = "BILLTOCUSTOMERNUMBER", right_on = "CUST_ACCOUNT_NUMBER",  how="left", suffixes = ("_1", "_2"))
renewals_df_active_2["CUST_ACCOUNT_NUMBER"] = renewals_df_active_2["CUST_ACCOUNT_NUMBER_1"].fillna(renewals_df_active_2["CUST_ACCOUNT_NUMBER_2"])
renewals_df_active_2 = renewals_df_active_2.drop(["BILLTOCUSTOMERNUMBER", "SHIPTOCUSTNUM","CUST_ACCOUNT_NUMBER_1","CUST_ACCOUNT_NUMBER_2"], axis=1)
renewals_df_active_2 = renewals_df_active_2.dropna()

renewals_df_active_2["STARTDATECOVERAGE"] =  pd.to_datetime(renewals_df_active_2["STARTDATECOVERAGE"])
renewals_df_active_2["RENEWALS_START_MONTH"] = pd.to_datetime(renewals_df_active_2["STARTDATECOVERAGE"]).dt.to_period("M").dt.to_timestamp()
renewals_df_active_2["RENEWALS_END_MONTH"] = pd.to_datetime(renewals_df_active_2["CONTRACT_END_DATE"]).dt.to_period("M").dt.to_timestamp()
renewals_df_active_2["RENEWALS_EARLIEST_MONTH"] = renewals_df_active_2.groupby(renewals_df_active_2["CUST_ACCOUNT_NUMBER"])["RENEWALS_START_MONTH"].transform("min")
renewals_df_active_2["RENEWALS_LATEST_MONTH"] = renewals_df_active_2.groupby(renewals_df_active_2["CUST_ACCOUNT_NUMBER"])["RENEWALS_END_MONTH"].transform("max")

#DNB Risk Breakdown
dnb_risk_df["ACCOUNT_NUMBER"] = dnb_risk_df["ACCOUNT_NUMBER"].astype("Int64").astype(str)
dnb_risk_df_active = dnb_risk_df.merge(l1_cust_active[["CUST_ACCOUNT_NUMBER"]], left_on = "ACCOUNT_NUMBER", right_on = "CUST_ACCOUNT_NUMBER",  how="inner")
dnb_risk_df_active = dnb_risk_df_active.drop("ACCOUNT_NUMBER", axis=1)

usage_latest["YYYYWK_Transformed"] = pd.to_datetime(usage_latest["YYYYWK"].apply(convert_yyyywk_to_date), errors = "coerce")

#Usage Japan Latest
usage_latest["CUST_ACCOUNT_NUMBER"] = usage_latest["CUST_ACCOUNT_NUMBER"].astype("Int64").astype(str)

usage_latest["YYYYWK_MONTH"] = pd.to_datetime(usage_latest["YYYYWK_Transformed"]).dt.to_period("M").dt.to_timestamp()

usage_latest_active = usage_latest.merge(l1_cust_active[["CUST_ACCOUNT_NUMBER"]], on="CUST_ACCOUNT_NUMBER",  how="inner")

# Merging all the active customers data frames i.e. Payments, Revenue, Transactions, contracts, contracts subline, contracts topline, renewals, snow inc, tech survey, loyalty survey, dnb risk and usage latest

merged_1 = p_r_t_merged.merge(contracts_sub_df_active, left_on = ["CUST_ACCOUNT_NUMBER", "MONTH"], right_on = ["CUST_ACCOUNT_NUMBER", "SUB_START_MONTH"], how="outer")

merged_1["MONTH"] = merged_1["MONTH"].fillna(merged_1["SUB_START_MONTH"])
merged_1 = merged_1.drop("SUB_START_MONTH", axis=1)
merged_2 = merged_1.merge(renewals_df_active_2, left_on = ["CUST_ACCOUNT_NUMBER","MONTH"], right_on = ["CUST_ACCOUNT_NUMBER","RENEWALS_START_MONTH"], how="outer")
merged_2["MONTH"] = merged_2["MONTH"].fillna(merged_2["RENEWALS_START_MONTH"])
merged_2 = merged_2.drop("RENEWALS_START_MONTH", axis=1)

merged_3 = merged_2.merge(dnb_risk_df_active, on="CUST_ACCOUNT_NUMBER", how="left")

merged_4 = merged_3.merge(usage_latest_active, left_on= ["CUST_ACCOUNT_NUMBER","MONTH"], right_on = ["CUST_ACCOUNT_NUMBER", "YYYYWK_MONTH"], how="outer")
merged_4["MONTH"] = merged_4["MONTH"].fillna(merged_4["YYYYWK_MONTH"])

to_drop_2 = ["CONTRACT_NUMBER", "CUST_PARTY_NAME", "CUSTOMER_NAME", "CONTRACT_END","JAROWINKLER_SIMILARITY(A.CUST_PARTY_NAME, B.CUSTOMER_NAME)"]

merged_4 = merged_4.drop(to_drop_2, axis=1)

merged_5 = merged_4.merge(l1_cust_active[["CUST_ACCOUNT_NUMBER", "CHURNED_FLAG"]], on = "CUST_ACCOUNT_NUMBER", how="inner")
merged_5["EARLIEST_DATE"] = merged_5[["RECEIPT_DATE", "DATE_INVOICE_GL_DATE", "TRX_DATE", "SLINE_START_DATE", "STARTDATECOVERAGE"]].min(axis=1)
merged_5["FINAL_EARLIEST_DATE"] = merged_5.groupby("CUST_ACCOUNT_NUMBER")["EARLIEST_DATE"].transform("min")

to_drop_3 = ["RECEIPT_DATE", "DATE_INVOICE_GL_DATE", "TRX_DATE", "SLINE_START_DATE", "SLINE_END_DATE", "SUB_END_MONTH","SLINE_STATUS", "SUB_EARLIEST_MONTH", "SUB_LATEST_MONTH", "STARTDATECOVERAGE", "CONTRACT_END_DATE", "RENEWALS_END_MONTH", "RENEWALS_EARLIEST_MONTH","RENEWALS_LATEST_MONTH", "YYYYWK_MONTH", "PERIOD", "CHURNED_FLAG", "EARLIEST_DATE"]

merged_5 = merged_5.drop(to_drop_3, axis=1)

date_col = ["MONTH","YYYYWK_Transformed", "FINAL_EARLIEST_DATE"]

merged_5[date_col] = merged_5[date_col].astype(str)

merged_5 = merged_5.drop_duplicates()

session.write_pandas(merged_5, "PS_DOCUWARE_RAW_DATA_PREDICTION", auto_create_table=True, overwrite = True)

# Data Preprocessing of Live customers

In [ ]:
#Fetching from previous SPROC output table
raw_df_active = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_RAW_DATA_PREDICTION").to_pandas()

non_ts_numeric_cols = ["PROBABILITY_OF_DELINQUENCY", "RICOH_CUSTOM_RISK_MODEL"]
non_ts_categorical_cols = ["OVERALL_BUSINESS_RISK", "PAYMENT_RISK_TRIPLE_A_RATING", "CONTRACT_LINE_ITEMS"]
columns_to_be_processed_later = non_ts_numeric_cols + non_ts_categorical_cols + ["CUST_ACCOUNT_NUMBER"]
finalized_df_ohe_to_process = raw_df_active.groupby("CUST_ACCOUNT_NUMBER")[columns_to_be_processed_later].first()


# Imputation for Non Time Series columns
pofd_median = finalized_df_ohe_to_process["PROBABILITY_OF_DELINQUENCY"].median()
finalized_df_ohe_to_process["PROBABILITY_OF_DELINQUENCY"] = finalized_df_ohe_to_process["PROBABILITY_OF_DELINQUENCY"].apply(lambda x:   float(pofd_median) if np.isnan(x) else x)

rcrm_mode = finalized_df_ohe_to_process["RICOH_CUSTOM_RISK_MODEL"].mode()
finalized_df_ohe_to_process["RICOH_CUSTOM_RISK_MODEL"] = finalized_df_ohe_to_process["RICOH_CUSTOM_RISK_MODEL"].apply(lambda x: float(rcrm_mode) if np.isnan(x) else x)

# One Hot Encoding for Categorical variables OVERALL_BUSINESS_RISK and PAYMENT_RISK_TRIPLE_A_RATING
finalized_df_ohe_to_process = finalized_df_ohe_to_process.reset_index(drop=True)

categorical_cols = ["OVERALL_BUSINESS_RISK", "PAYMENT_RISK_TRIPLE_A_RATING"]
fill_value = "UNK"

for col in categorical_cols:
    finalized_df_ohe_to_process[col].fillna(fill_value, inplace=True)
    finalized_df_ohe_to_process[col] = finalized_df_ohe_to_process[col].str.replace(" ", "_", regex=False)
    
    if col == "OVERALL_BUSINESS_RISK":
        col_abreviation = "obr_"
    else:
        col_abreviation = "prtar_"
        
    le_ohe = LabelEncoder()
    ohe = OneHotEncoder(handle_unknown = "ignore")
    enc_train = le_ohe.fit_transform(finalized_df_ohe_to_process[col]).reshape(finalized_df_ohe_to_process.shape[0],1)
    ohe_train = ohe.fit_transform(enc_train)
    le_ohe_name_mapping = dict(zip(le_ohe.classes_, le_ohe.transform(le_ohe.classes_)))
    
    enc_train = finalized_df_ohe_to_process[col].map(le_ohe_name_mapping).ravel().reshape(-1,1)
    enc_train[np.isnan(enc_train)] = 9999

    cols = [col_abreviation + str(x) for x in le_ohe_name_mapping.keys()]
    finalized_df_ohe_to_process = pd.concat([finalized_df_ohe_to_process.reset_index(), pd.DataFrame.sparse.from_spmatrix(ohe_train, columns = cols)], axis = 1).drop(["index"], axis=1)
    finalized_df_ohe_to_process.drop([col], axis = 1, inplace=True)

columns_to_be_droped = non_ts_categorical_cols+non_ts_numeric_cols
raw_df_active.drop(columns_to_be_droped, axis=1, inplace=True)

# Target Encoding for CONTRACT_LINE_ITEMS
finalized_df_ohe_to_process["CONTRACT_LINE_ITEMS"].fillna("NA", inplace=True)
finalized_df_ohe_to_process["CONTRACT_LINE_ITEMS"] = finalized_df_ohe_to_process["CONTRACT_LINE_ITEMS"].str.replace(r"\\d+x ", "", regex=True)

for i, row in finalized_df_ohe_to_process.iterrows():
    t = row["CONTRACT_LINE_ITEMS"]
    arr = t.split("-")
    arr = [x.strip() for x in arr]
    arr_s = sorted(arr)
    key = "-".join([s for s in arr_s])
    finalized_df_ohe_to_process.loc[i, "CONTRACT_LINE_ITEMS"] = key

stage_path = "@PS_DOCUWARE_CHURN/ps_docuware_target_encoder_object"

X_train = finalized_df_ohe_to_process.copy()

import_dir = sys._xoptions.get("snowflake_import_directory")
enc_file = os.path.join('/tmp', 'ENC_CURRENT.joblib.gz')
session.file.get("@PS_DOCUWARE_CHURN/ps_docuware_target_encoder_object/ENC_CURRENT.joblib.gz", '/tmp')
enc = load(enc_file)

X_train = X_train.reindex(columns = enc.feature_names_in_, fill_value=0)
X_train_encoded = enc.transform(X_train)

# Imputation for Time Series columns
ts_columns = ["CUST_ACCOUNT_NUMBER", "YYYYWK", "DOCUMENTS_OPENED", "USED_STORAGE__MB", "INVOICE_REVLINE_TOTAL", "ORIGINAL_AMOUNT_DUE", "FUNCTIONAL_AMOUNT"]
raw_df_active["transformed_YYYYWK"] = raw_df_active["MONTH"].apply(convert_date_to_yyyywk)
#Impute missing YYYYWK with equivalent MONTH
raw_df_active["YYYYWK"].fillna(raw_df_active["transformed_YYYYWK"], inplace=True)
raw_df_active.drop("transformed_YYYYWK", axis=1, inplace=True)
ts_df = raw_df_active[ts_columns]
ts_df = ts_df[ts_df['YYYYWK'].notna()]
ts_df['YYYYWK'] = ts_df['YYYYWK'].astype(int)
ts_df['CUST_ACCOUNT_NUMBER'] = ts_df['CUST_ACCOUNT_NUMBER'].astype(int)
ts_df.rename(columns={'USED_STORAGE__MB':'USED_STORAGE_MB'},inplace=True)
ts_df_sorted = ts_df.sort_values(['CUST_ACCOUNT_NUMBER','YYYYWK']).drop_duplicates()

only_features = ts_df_sorted.copy()
only_features = only_features.fillna(0)

#ts_comprehensive_df = session.sql("SELECT * FROM RAC_RAPID_DEV.RUS_AIML.PS_DOCUWARE_PREDICTION_TS_DF_ONLY").to_pandas()
#training_set_ts_df = session.sql("SELECT * FROM RAC_RAPID_DEV.RUS_AIML.PS_DOCUWARE_ONLY_TS_DF").to_pandas()


#ts_comprehensive_df_active = time_series_ts_fresh_features(only_features)


# Store Live Customers' TimeSeries data before TSFRESH into DB

In [ ]:
session.write_pandas(only_features, "PS_DOCUWARE_LIVE_RAW_DATA_BEFORE_TSFRESH", auto_create_table=True, overwrite = True)

# Feature Engineering using TSFRESH

In [ ]:
ts_comprehensive_df_active = engineer_timeseries_cols_using_tsfresh_for_live_customers(only_features)
ts_comprehensive_df_active = ts_comprehensive_df_active.replace([np.inf, -np.inf], 0)
ts_comprehensive_df_active = ts_comprehensive_df_active.fillna(0)
training_set_ts_df = ts_comprehensive_df_active.copy()
same_features = training_set_ts_df.columns

In [ ]:
#same_features
#ts_comprehensive_df_active = fts_comprehensive_df_active

# Store LIVE Customers' TSFRESH time series generated features

In [ ]:
ts_comprehensive_df_active.shape
session.write_pandas(ts_comprehensive_df_active, "PS_DOCUWARE_LIVE_CUSTOMER_FEATURES", auto_create_table=True, overwrite = True)

In [ ]:
# #comprehensive_imputed_df_active['FINAL_EARLIEST_DATE']
# temp = raw_df_active[['CUST_ACCOUNT_NUMBER', 'FINAL_EARLIEST_DATE']].drop_duplicates()
# temp.shape, ts_comprehensive_df_active.shape, X_train_encoded.shape

# Data preprocessing to prepare it for inferencing 

In [ ]:
ts_comprehensive_df_active = ts_comprehensive_df_active.reindex(columns=same_features, fill_value=0)
ts_comprehensive_df_active["CUST_ACCOUNT_NUMBER"] = ts_comprehensive_df_active["CUST_ACCOUNT_NUMBER"].astype(int)

X_train_encoded["CUST_ACCOUNT_NUMBER"] = X_train_encoded["CUST_ACCOUNT_NUMBER"].astype(int)
df_earliest_date = raw_df_active[['CUST_ACCOUNT_NUMBER', 'FINAL_EARLIEST_DATE']].drop_duplicates()
df_earliest_date["CUST_ACCOUNT_NUMBER"] = df_earliest_date["CUST_ACCOUNT_NUMBER"].astype(int)

comprehensive_imputed_df_active = pd.merge(ts_comprehensive_df_active, X_train_encoded, on="CUST_ACCOUNT_NUMBER", how="inner")
comprehensive_imputed_df_active = pd.merge(comprehensive_imputed_df_active, df_earliest_date, on="CUST_ACCOUNT_NUMBER", how="inner")
comprehensive_imputed_df_active['CREATION_DATE'] = pd.Timestamp.today().date()

#comprehensive_imputed_df_active['DAYS_ELAPSED_TILL_DATE'] = (pd.to_datetime(comprehensive_imputed_df_active['CREATION_DATE']) - pd.to_datetime(comprehensive_imputed_df_active['FINAL_EARLIEST_DATE'])).dt.days

#stratified_df_active = pd.DataFrame()

# Get CLUSTER Information in the dataframe
#label_count=1
# for i in range(365, 1461, 365):
#     XX = comprehensive_imputed_df_active[((i-365) < comprehensive_imputed_df_active["DAYS_ELAPSED_TILL_DATE"]) & (comprehensive_imputed_df_active["DAYS_ELAPSED_TILL_DATE"] <= i)]
#     #XX["SAMPLE"] = label_count
#     stratified_df_active = pd.concat([stratified_df_active, XX], axis=0, ignore_index=True)
#     #label_count+=1
   
#     if i==1460:
#         XX = comprehensive_imputed_df_active[1460 < comprehensive_imputed_df_active["DAYS_ELAPSED_TILL_DATE"]]
#         #XX["SAMPLE"] = label_count
#         stratified_df_active = pd.concat([stratified_df_active, XX], axis=0, ignore_index=True)

comprehensive_imputed_df_active_with_cluster = comprehensive_imputed_df_active.copy()
# comprehensive_imputed_df_active_with_cluster = stratified_df_active.copy()

for col in comprehensive_imputed_df_active_with_cluster.columns:
    if pd.api.types.is_sparse(comprehensive_imputed_df_active_with_cluster[col]):
        comprehensive_imputed_df_active_with_cluster[col] = comprehensive_imputed_df_active_with_cluster[col].sparse.to_dense()

#import_dir = sys._xoptions.get("snowflake_import_directory")        
model_file = os.path.join('/tmp', 'xgb.joblib.gz')
session.file.get("@PS_DOCUWARE_CHURN/ps_docuware_churn_model/xgb.joblib.gz", "/tmp")
best_model = load(model_file)

#input_features = comprehensive_imputed_df_active_with_cluster.drop(["CUST_ACCOUNT_NUMBER", "CREATION_DATE", "DAYS_ELAPSED_TILL_DATE"], axis=1)
input_features = comprehensive_imputed_df_active_with_cluster.drop(["CUST_ACCOUNT_NUMBER", "CREATION_DATE"], axis=1)

input_features = convert_to_float(input_features)
input_features = input_features.reindex(columns=best_model.feature_names_in_, fill_value=0)



# Store features of Live Customers into Staging

In [ ]:
df_live_customers = session.create_dataframe(input_features)
#df_live_customers.write.copy_into_location("@PS_DOCUWARE_CHURN/df_live_customers_21Apr2025.csv", file_format_type="csv", format_type_options={"COMPRESSION": "NONE"}, header=True, overwrite=True )
        

# Store features of Live Customers into DB

In [ ]:
session.write_pandas(df_live_customers, "PS_DOCUWARE_LIVE_CUSTOMER_FEATURES", auto_create_table=True, overwrite = True)

In [ ]:
import shap


# Inferencing for Live customers to predict the churn duration

In [ ]:
y_pred_transformed = best_model.predict(input_features)
remaining_months = y_pred_transformed/30
#lifespan_months = np.exp(y_pred_transformed)

final_predicted_df = pd.DataFrame({
    "CUST_ACCOUNT_NUMBER": comprehensive_imputed_df_active["CUST_ACCOUNT_NUMBER"].astype(str),
    "MONTHS_REMAINING": remaining_months
})



In [ ]:
final_predicted_df.head(50)

In [ ]:
raw_df_active_1 = raw_df_active[['CUST_ACCOUNT_NUMBER', 'FINAL_EARLIEST_DATE']].drop_duplicates()

final_predicted_df = final_predicted_df.merge(raw_df_active_1, on='CUST_ACCOUNT_NUMBER', how='inner')

final_predicted_df['CREATION_DATE'] = pd.Timestamp.today().date()
final_predicted_df = final_predicted_df.rename({'FINAL_EARLIEST_DATE':'EARLIEST_START_DATE'})


final_predicted_df['DAYS_ELAPSED'] = (pd.to_datetime(final_predicted_df['CREATION_DATE']) - pd.to_datetime(final_predicted_df['FINAL_EARLIEST_DATE'])).dt.days
final_predicted_df['MONTHS_ELAPSED'] = (pd.to_datetime(final_predicted_df['CREATION_DATE']) - pd.to_datetime(final_predicted_df['FINAL_EARLIEST_DATE'])).dt.days/30
#final_predicted_df['MONTHS_REMAINING'] = final_predicted_df['LIFESPAN_MONTHS'] #- final_predicted_df['MONTHS_ELAPSED']
final_predicted_df['LIFESPAN_MONTHS'] = final_predicted_df['MONTHS_REMAINING'] + final_predicted_df['MONTHS_ELAPSED']
final_predicted_df.head(100)

# Store Prediction for Live customers into DB

In [ ]:
session.write_pandas(final_predicted_df, "PS_DOCUWARE_PREDICTION_DATA_NEW", auto_create_table=True, overwrite = True)
session.sql("INSERT INTO rus_aiml.PS_DOCUWARE_PREDICTION_DATA_ARCHIVE SELECT * FROM rus_aiml.PS_DOCUWARE_PREDICTION_DATA_NEW WHERE CREATION_DATE not in (select creation_date from rus_aiml.PS_DOCUWARE_PREDICTION_DATA_ARCHIVE)")
#session.sql("INSERT INTO PS_DOCUWARE_PREDICTION_DATA_ARCHIVE SELECT * FROM PS_DOCUWARE_PREDICTION_DATA_NEW WHERE CREATION_DATE != CURRENT_DATE()")

In [ ]:
!pip install shap



In [ ]:
model_file = os.path.join('/tmp', 'xgb.joblib.gz')
session.file.get("@PS_DOCUWARE_CHURN/ps_docuware_churn_model/xgb.joblib.gz", "/tmp")
best_model = load(model_file)
#live_features = session.sql("SELECT * FROM RUS_AIML.PS_DOCUWARE_LIVE_CUSTOMER_FEATURES").to_pandas()
explainer = shap.Explainer(best_model.predict, input_features)
shap_values = explainer.shap_values(input_features)

In [ ]:
shap_values.shape

In [ ]:
shap.summary_plot(shap_values)

In [ ]:
shap.summary_plot(shap_values, plot_type='violin')

In [ ]:
feature_importance = global_shap_importance(best_models, input_features)

In [ ]:
feature_importance_top40 = feature_importance.head(40)

# Plot of Shap to get top 40 important features for Live customer's churn prediction

In [ ]:
# Sample data
y = feature_importance_top40['importance']
x = feature_importance_top40['features']

# Create a figure and a set of subplots with a specified size
fig, ax = plt.subplots(figsize=(40, 30))  # width=12 inches, height=8 inches

# Plot the data on the axes
ax.plot(x, y)

# Add labels and title
ax.set_xlabel('importance score')
ax.set_ylabel('Features')
ax.set_title('Graph of Top 40 Feature vs Importance While Predicting Input of 405 customers')
plt.xticks(x, x, rotation=45, ha='right')
#plt.savefig('C:/My Documents/work/Projects/Short Term Goal 2 - Customer Churn Prediction/ML Modeling Phase/Shap_Top_30_Importance_Features.png')
# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

x = [1, 2, 3, 4, 5]
y = [2, 4, 1, 3, 5]

plt.plot(x, y, marker='o', linestyle='-', color='b', label='Data Points')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('My Graph')
plt.legend()

graph_file = os.path.join("/tmp", "graph_new.jpg")
plt.savefig(graph_file)
session.file.put(graph_file, "@PS_DOCUWARE_CHURN/graph_new.jpg",overwrite=True)

plt.show()

In [ ]:
 PUT /tmp/graph_new.jpg @PS_DOCUWARE_CHURN/ OVERWRITE=TRUE;